# Deep Reinforcement Learning, due 5/12/2022.

In [ ]:
import torch as th
import numpy as np
import gym
import matplotlib.pyplot as plt

In [ ]:
env = gym.make("CartPole-v1")

## Exercise 1: Implement the Fitted Q-Iteration Algorithm

### Task A: Implement a Q-Network class: a Q-function S X A-> |R approximated by a neural network.

In [ ]:
class Q(th.nn.Module):
    def __init__(self, env, hidden_size=64, hidden_layers = 2, activ=th.nn.ReLU, optimizer=th.optim.Adam):
        super().__init__()
        self.env = env
        
        self.layers = ...
        
        self.optimizer = optimizer(self.parameters())
        self.optimizer.zero_grad()

    def forward(self, states_actions ):
        ...
        return q_s_a
    
    def get_best_action(self, state):
        with torch.no_grad():
            ...
        return best_action

In [ ]:
#test your Q network
q = Q(test_environment)

#data collection
batch_size = 32
batch = np.zeros((batch_size, 2 * env.observation_space.shape[0] + 3)) #batch size X s + a + r + snext + done
s = env.reset()
for i in range(batch_size):
    s = env.observation_space.sample()
    env.state = s
    a = env.action_space.sample()
    snext, r, done, infos = env.step(a)
    transition = np.array([*s, a, r, *snext, int(done)])
    batch[i] = transition
    if done:
        s = env.reset()
    else:
        s = snext
    
#convert batch of np.arrays to tensors
batch = th.FloatTensor(batch)
states_actions_tensors = batch[:, :env.observation_space.shape[0] + 1]
#Check the output size of your Q-network
assert q.forward(states_actions_tensors).size(1) == 1

### Task B: write a function to fit your Q-network on a batch of transitions.

In [ ]:
MSE = th.nn.MSELoss()
def fit(Q, batch, gamma = 0.99):
    predictions = ...
    targets = ...
    loss = MSE(prefictions, targets)
    loss.backward()
    Q.optimizer.zero_grad()   
    return loss.item()

### Task C: test fitted Q-Iteration on CartPole and Acrobot and extract some policies.

In [ ]:
cartpole = gym.make("CartPole-v1")
acrobot = gym.make("Acrobot-v1")

def fittedQiter(env, iterations = 100, batch_size = 32):
    qnet = Q(env)
    losses = []
    
    for it in range(iterations):
        #1. get a batch
        batch = np.zeros((batch_size, 2 * env.observation_space.shape[0] + 3)) 
        ...
        #2. fit Q-network on batch
        loss = fit(qnet, batch)
        losses.append(loss)
    
    return qnet, losses


def policy(qnet, s):
    ...
    return action.item()

def evaluate(env, qnet, nb_episode = 100):
    scores = []
    for i in range(nb_episode):
        cumulative_reward = 0
        s = env.reset()
        done = False
        while not done:
            ...
        scores.append(cumulative_reward)
    return scores

#plot losses and analyse evaluations

## Exercise 2: Implement the Deep Q-Networks algorithm with a replay buffer and study its performance on Acrobot and Cartpole.